# Выявление особенностей словоупотребления в текстах Л. Н. Толстого методами дистрибутивной семантики

** TO DO **

- обучить сегментатор поточнее

In [4]:
import os
import re
from lxml import etree
from pymystem3 import Mystem
import string
import nltk
#from langdetect import detect
#import langid
from datetime import datetime
mystem = Mystem()

## Сегментатор

In [5]:
language_punkt_vars = nltk.tokenize.punkt.PunktLanguageVars
language_punkt_vars.sent_end_chars=('.','?','!','»','…')
with open('.\\data\\traindata.txt', 'r', encoding='utf-8') as f:
    traindata = f.read()
trainer = nltk.tokenize.punkt.PunktTrainer(traindata, language_punkt_vars)
trainer.INCLUDE_ALL_COLLOCS = True 
trainer.INCLUDE_ABBREV_COLLOCS = True
params = trainer.get_params()
sbd = nltk.tokenize.punkt.PunktSentenceTokenizer(params)

  Abbreviation: [1.4849] ст
  Abbreviation: [4.0363] н
  Abbreviation: [1.4849] ул
  Abbreviation: [8.0726] м
  Abbreviation: [5.4625] тыс
  Abbreviation: [5.9395] т.п
  Abbreviation: [19.3033] см
  Abbreviation: [0.5463] стр
  Abbreviation: [1.4849] кн
  Abbreviation: [4.0363] д
  Abbreviation: [1.0925] руб
  Abbreviation: [4.0363] п
  Abbreviation: [0.6029] матф
  Abbreviation: [4.0363] г
0.000380517503805175 0.007662835249042145 0.0 5256 261 2 2
0.000380517503805175 0.007662835249042145 0.0 5256 261 2 2
0.0030441400304414 0.007662835249042145 0.0028028028028028026 5256 261 16 2
0.009512937595129375 0.038314176245210725 0.008008008008008008 5256 261 50 10
0.0030441400304414 0.007662835249042145 0.0028028028028028026 5256 261 16 2
0.001141552511415525 0.007662835249042145 0.0008008008008008008 5256 261 6 2
0.0036149162861491627 0.007662835249042145 0.0034034034034034033 5256 261 19 2
0.00076103500761035 0.007662835249042145 0.0004004004004004004 5256 261 4 2
0.0015220700152207 0.02298

## Данные

На основе 90-томного собрания сочинений

Namespace тегов

In [6]:
tei_ns = {'tei': 'http://www.tei-c.org/ns/1.0'}

Знаки пунктуации для удаления

In [7]:
string.punctuation = string.punctuation.replace('-','')
string.punctuation

'!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'

## Функции предобработки

#### Функция для парсинга одного файла.

- ** Вход : ** имя файла
- ** Дополлнительные сведения : ** теги для удаления и исключения, пространство имен тегов получает из пространства имен (области видимости) выше по уровню
- ** Выход : ** нужный текст

In [8]:
def get_one_writing(filename):
    global tei_ns
    root = etree.parse(filename)#.getroot()
    for i in tags_for_deletion:
        for j in root.findall(i, tei_ns):
            parent = j.find("..")
            parent.remove(j)
    for i in tags_to_strip:
        etree.strip_tags(root, i)
    return root.find(".//tei:text", tei_ns).text.strip()

#### Функция для препроцессинга текста

- ** Вход : ** текст, файл записи
- ** Дополнительная информация : ** сегментатор, Mystem - глобальные переменные
- ** Выход : ** количество предложений (для вывода и отслеживания)


** Ход преобразований **

1. Исключение множественных пробелов и переносов строки
2. Деление на предложения
3. Объединение с разделителем в виде $$, так как mystem быстрее работает при одном вызове на большой файл, чем на маленьких частях. И такой знак не встречается в текстах
4. Удаляем [], в которых в текстах восстановлены сокращения
5. Лемматизируем с помощью mystem
6. Удаляем некириллические символы (удаляем нерусский текст, знаки препинания (кроме дефисов), оставляем наши разделители
7. Удаялем возможные множественные дефисы
8. Удаляем остатки от слов типа 8-го (-го).
9. Восстанавливаем разбиение на предложения
10. Если от строки осталось более 25 символов (удаляем слишком маленькие предложения и строки, которые у нас отделяются со ссылками типа (Лука, VII), то записываемв файл.

In [9]:
def preprocessing(text, f):
    global sbd, mystem
    text = re.sub('[\n\s]+',' ', text)
    lng = len(sbd.sentences_from_text(text, realign_boundaries=True))
    text = ' $$ '.join(sbd.sentences_from_text(text, realign_boundaries=True))
    text = text.replace('[','').replace(']','')
    text = ''.join(mystem.lemmatize(text.strip()))
    text = re.sub('[^а-яА-Я\-$ ]','', text)
    text = re.sub('[-]{2,}','', text)
    text = re.sub('[^а-яА-Я]-.*? ',' ', text)
    text = text.split('$$')
    for i in text:
        if len(i.strip())>25:
            f.write(i.strip()+'\n')
    return lng

In [10]:
with open ('1.txt', 'w') as f:
    preprocessing ('Я 1-го числа никак не приду к вам.\n\nЗасим удаляюсь.', f)

#### Функция объединяющая преобразования

- ** Вход : ** путь к папке с данными, тэги для удаления, теги для опущения, имя файла записи, параметр печати логов

** Печать **

- если установлен флажок печати, то печатает: количество предложений в тексте, размер файла (для проверки), время обработки файла, имя файла
- общее время работы

In [11]:
def universal_preprocessing(path, 
                            tags_for_deletion, 
                            tags_to_strip, 
                            filename, printing=True):
    #tags_for_deletion = tags_for_deletion
    #tags_to_strip = tags_to_strip
    #mystem = Mystem()
    start_gen = datetime.now()
    with open (filename, 'w', encoding='utf-8') as f:
        for root, dirs, files in os.walk (path):
                for fl in files :
                    start = datetime.now() 
                    lng = preprocessing(get_one_writing(root+'\\'+fl), f)
                    if printing:
                        print ('{}\t{}\t{}\t{}'.format(lng, os.path.getsize(root+'\\'+fl)//1024, 
                                              datetime.now()-start,
                                              fl))
    print ('\n\ntotal time:\t\t {}'.format(datetime.now()-start_gen))
    

### Произведения

Удаляем ненужные комментарии (заметки, пояснения, старые формы, где есть исправления, элементы таблицы). Убираем теги, которые не несут информации, чтобы объединить в единый массив текста.

In [12]:
tags_for_deletion  = ['.//tei:note','.//tei:add','.//tei:sic','.//tei:orig',
                      './/tei:head','.//tei:del','.//tei:figure','.//tei:table']

tags_to_strip = ['{http://www.tei-c.org/ns/1.0}pb', '{http://www.tei-c.org/ns/1.0}hi', '{http://www.tei-c.org/ns/1.0}corr',
                '{http://www.tei-c.org/ns/1.0}reg', '{http://www.tei-c.org/ns/1.0}choice', '{http://www.tei-c.org/ns/1.0}p',
                '{http://www.tei-c.org/ns/1.0}div', '{http://www.tei-c.org/ns/1.0}lb']

НЕ ЗАПУСКАТЬ!!!

In [ ]:
path = '.\\data\\Final\\'
universal_preprocessing(path, tags_for_deletion, tags_to_strip, 'writings.txt')

938	125	0:00:02.182899	Abrégé de l’Evangile 24.xml
21	8	0:00:01.690339	AMOUR DE LA PATRIE 1.xml
70	37	0:00:01.979403	Carthago delenda est 39.xml
87	32	0:00:01.801238	Carthago delenda est. Черновое 39.xml
322	95	0:00:02.451469	Letter to a Hindoo (Письмо к индусу) 37.xml
44	13	0:00:01.980461	NOTES SUR LE SECOND CHAPITRE DES «CARACTÈRES» DE LA BRUYÈRE 1.xml
28	13	0:00:02.084049	The hostelry (Гостиница) 37.xml
19	19	0:00:02.041133	[Carthago delenda est] («Жизнь, та форма жизни»...) 27.xml
9	5	0:00:01.725240	[LE PRÉSENT, LE PASSÉ ET LE FUTUR] 1.xml
9	9	0:00:01.620498	[«Давно позабыл я о счастьи ...»] 1.xml
49	29	0:00:01.531272	[«Дядинька Жданов и кавалер Чернов»] 3.xml
60	42	0:00:01.624674	[«Ежели бы человек не желал…»] 1.xml
18	12	0:00:01.594855	[«Ей, Марьяна, брось работу!..»] 1.xml
37	22	0:00:01.562201	[«Знание музыки - субъективное и объективное...»] 1.xml
728	330	0:00:02.259206	[«Начала» романа «Сто лет»] 17.xml
370	203	0:00:02.007993	[«Начала» романа времен Петра I. Азовские походы] 1

### Письма

Добавляются теги <opener> и <closer> как элементы письма, которые не относятся к непосредственному текксту.

In [14]:
tags_for_deletion  = ['.//tei:note','.//tei:add','.//tei:sic','.//tei:orig',
                      './/tei:head','.//tei:del','.//tei:figure','.//tei:table',
                      './/tei:opener', './/tei:closer']

tags_to_strip = ['{http://www.tei-c.org/ns/1.0}pb', '{http://www.tei-c.org/ns/1.0}hi', '{http://www.tei-c.org/ns/1.0}corr',
                '{http://www.tei-c.org/ns/1.0}reg', '{http://www.tei-c.org/ns/1.0}choice', '{http://www.tei-c.org/ns/1.0}p',
                '{http://www.tei-c.org/ns/1.0}div', '{http://www.tei-c.org/ns/1.0}lb','{http://www.tei-c.org/ns/1.0}body']

In [16]:
path = '.\\data\\LETTERS\\'
for root, dirs, files in os.walk (path):
    for fl in files :
        with open (root+'\\'+fl, 'r', encoding='utf-8') as f:
            s = f.read()
            s = re.sub('<note.*?>', '<note>', s)
            s = re.sub('xml:id=".*?"','',s)
        with open (root+'\\'+fl,'w', encoding='utf-8') as f:
            f.write(s)

In [17]:
path = '.\\data\\LETTERS\\'
universal_preprocessing(path, tags_for_deletion, tags_to_strip, 'letters.txt', printing=False)



total time:		 4:23:10.430283


### Дневники

In [9]:
tags_for_deletion  = ['.//tei:note','.//tei:add','.//tei:sic','.//tei:orig',
                      './/tei:head','.//tei:del','.//tei:figure','.//tei:table', './/tei:opener']

tags_to_strip = ['{http://www.tei-c.org/ns/1.0}pb', '{http://www.tei-c.org/ns/1.0}hi', '{http://www.tei-c.org/ns/1.0}corr',
                '{http://www.tei-c.org/ns/1.0}reg', '{http://www.tei-c.org/ns/1.0}choice', '{http://www.tei-c.org/ns/1.0}p',
                '{http://www.tei-c.org/ns/1.0}div', '{http://www.tei-c.org/ns/1.0}lb','{http://www.tei-c.org/ns/1.0}body']

In [14]:
path = '.\\data\\diaries\\'
universal_preprocessing(path, tags_for_deletion, tags_to_strip, 'diaries.txt')

845	981	0:00:02.328109	46.xml
522	698	0:00:02.080954	47.xml
2673	268	0:00:03.071816	48.xml
4699	425	0:00:04.272224	49.xml
6190	778	0:00:05.901006	50.xml
3502	441	0:00:03.794691	51.xml
3827	612	0:00:04.737981	52.xml
4718	897	0:00:06.293261	53.xml
1582	752	0:00:02.762088	54.xml
2210	1117	0:00:03.462921	55.xml
2574	639	0:00:03.810186	56.xml
2833	733	0:00:03.853652	57.xml
2451	568	0:00:02.957235	58.xml


total time:		 0:00:49.328126


## Соединить источники

In [10]:
files = ['writings.txt', 'letters.txt', 'diaries.txt']
with open ('full_collection.txt', 'w',encoding = 'utf-8') as outp:
    for file in files:
        with open (file, 'r', encoding = 'utf-8') as inp:
            for line in inp:
                outp.write(line)

## Word2Vec

In [11]:
import gensim

C:\Users\Glaz\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Так как данные имеют большой размер, подаем на вход объект, считываемыц из файла (с возможностью итерации без хранения в памяти)

In [12]:
sentences = gensim.models.word2vec.LineSentence('full_collection.txt')

In [13]:
model = gensim.models.Word2Vec(sentences, min_count=1, sorted_vocab=1)

In [14]:
print(model)

Word2Vec(vocab=58335, size=100, alpha=0.025)


In [15]:
print(model.wv.vocab.keys())

dict_keys(['евангелие', 'быть', 'откровение', 'истинный', 'благо', 'иисус', 'христос', 'сын', 'бог', 'воля', 'это', 'жизнь', 'и', 'весь', 'человек', 'жалеть', 'который', 'не', 'знать', 'он', 'учить', 'народ', 'говорить', 'счастливый', 'тот', 'кто', 'пресыщать', 'богатый', 'веселый', 'славный', 'среди', 'но', 'нищий', 'бедный', 'смирять', 'плакать', 'кроткий', 'искать', 'истина', 'милосердный', 'чистый', 'сердце', 'миротворец', 'изгонять', 'за', 'правда', 'они', 'даже', 'когда', 'обижать', 'преследовать', 'радоваться', 'мир', 'вы', 'оскорблять', 'гнать', 'так', 'же', 'пророк', 'горе', 'пресыщенный', 'смеяться', 'получать', 'что', 'ожидать', 'бедствие', 'этот', 'смиренный', 'ваш', 'счастие', 'в', 'бедность', 'смирение', 'ничто', 'мочь', 'нарушать', 'если', 'чувствовать', 'как', 'соль', 'потерять', 'свой', 'сила', 'должный', 'то', 'себя', 'свеча', 'закрывать', 'она', 'бесполезный', 'никто', 'освещать', 'светить', 'чтобы', 'видеть', 'прославлять', 'отец', 'на', 'небо', 'давать', 'мой', 'де